In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Endpoint 1

In [61]:
def developer(desarrollador: str):
    resp= ['Año', 'Cantidad de items', 'Contenido Free (%)'] #Creamos las etiquetas con las que vamos a hacer el diccionario de salida
    l = [] #Lista que será la salida de la funcion
    df = pd.read_parquet('endpoint1', engine='auto') #Leemos el Dataset
    c = df[(df['developer'] == desarrollador.lower())] #Lo filtramos según el valor ingresado
    f = c.groupby(['release_date'])['free'].sum().reset_index(name='free') #Sumamos la cantidad de juegos Free to Play por año
    c = c.groupby(['release_date'])['developer'].count().reset_index(name="count") #Contamos la cantidad de items del desarrollador por año
    c['free'] = f['free'] #Anexamos la cantidad de juego free que tiene por año la empresa
    
    for i in range(len(c)): #Ciclo for donde se extraen los valores de los resultados obtenidos y los ponemos en la primera lista
        d = [c['release_date'][i], c['count'][i], round(c['free'][i]*100 / c['count'][i])]
        z = dict(zip(resp,d))
        l.append(z)
    
    r = pd.DataFrame(l).to_dict(orient='records') #Pasamos la lista a diccionario para que FastAPI pueda reconocer la salida

    return "No se encontraron datos" if len(l) == 0 else r

In [62]:
developer('kotoshiro')

[{'Año': '2018', 'Cantidad de items': 1, 'Contenido Free (%)': 0}]

# Endpoint 2

In [56]:
def  userdata(User_id: str):
    df = pd.read_parquet('endpoint2', engine='auto') #Leemos el dataset

    if df[(df['user_id'] == User_id.lower())].empty == True: #Si el valor ingresado no arroja resultado, termina la función
        response = "No se encontraron datos"
    
    else:        
        df = df[(df['user_id'] == User_id.lower())] #Caso contrario, se filtra el dataframe según el valor ingresado

        if df[df['playtime_forever'] > 0].count().empty == True: #Si no se encuentan valores, defina la variable en 0
            jugados = 0
        else:
            jugados = df[df['playtime_forever'] > 0].count()[0] #Caso contrario, obtiene la cantidad de juegos jugados por el usuario

        if df.groupby('user_id').sum('price').empty == True: #Si no se encuentran valores, asigna 0 en la variable
            dinero = 0
        else:
            dinero = df.groupby('user_id').sum('price')['price'][0] #Caso contrario, se obtiene la cantidad de dinero gastado por el usario

    
    
        if df['rec_games'].iloc[0] == 0: # Si no recomendó ningún juego, asigna 0 a la variable
            porcentaje = 0
        else:
            porcentaje = df['rec_games'].iloc[0] * 100 / jugados #Caso contrario, saca el porcentaje de juegos recomendados sobre juegos jugados
    

        response = {'Usuario' : User_id.title(), 'Dinero gastado' : dinero, '% recomendación': porcentaje,
                'Cantidad de items': df['items_count'].iloc[0]}
    
    return response

In [60]:
userdata('sp3ctre')

{'Usuario': 'Sp3Ctre',
 'Dinero gastado': 41.95,
 '% recomendación': 160.0,
 'Cantidad de items': 571.0}

# Endpoint 3

In [54]:
def UserForGenre(genre: str):
    genero = f"Usuario con más horas jugadas para Género {genre.title()}" # Creamos etiqueta de salida
    df = pd.read_parquet('endpoint3', engine='auto') #Leemos dataset

    c = df[(df['genres'].str.contains(genre.lower()))] #filtramos por el genero ingresado que está dentro de las etiquetas de la columna
    c.drop(columns='genres', inplace=True) #Borramos la columna de generos para que agrupe mas facil a continuación
    max = c.groupby(['user_id'])['playtime_forever'].sum().sort_values(ascending=False).head(1).index[0] #Asignamos a la variable el nombre del usuario que más horas jugó al genero
    c = c[(c['user_id'] == max)].groupby(['release_date']).sum(['playtime_forever']).reset_index(names= 'anio') # Sumamos las horas jugadas por el usuario agrupado por año
    
    l = [] #creamos lista 
    for i in range(len(c)): #con un ciclo for iteramos sobre los resultados obtenidos y los ponemos en la lista
            d = {"Año": c['anio'].tolist()[i], "Horas": c['playtime_forever'].tolist()[i]}
            l.append(d)

    response = {genero : max, "Horas jugadas" : l} #Generamos la respuesta
        
    return response


In [55]:
UserForGenre('strategy')

C:\Users\Javo\AppData\Local\Temp\ipykernel_19404\1084552427.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c.drop(columns='genres', inplace=True) #Borramos la columna de generos para que agrupe mas facil a continuación


{'Usuario con más horas jugadas para Género Strategy': 'spludge',
 'Horas jugadas': [{'Año': '2015', 'Horas': 260155}]}

# Endpoint 4

In [51]:
def best_developer_year(año : int):
    df = pd.read_parquet('endpoint45', engine='auto') #Leemos el dataset


    df.drop(columns='sent_neg') #Sacamos la columna que no usamos
    df = df[(df['release_date'] == año)] #Filtramos el dataframe según el dato ingresado
    if df.empty: #Si no hay resultados, termina la función
        return "No se encontraron datos"
    df = df.groupby(['developer', 'release_date']).sum(['sent_pos', 'recommend_True']).sort_values(by='recommend_True',ascending=False).head(3) #Sumamos las recomendaciones y reviews positivas y las agrupamos por desarrollador y año

    response = {"Puesto 1": df.index[0][0].title(), "Puesto 2": df.index[1][0].title(), "Puesto 3": df.index[2][0].title()} #Extraemos los 3 con mayor cantidad de recomendaciones y reviews positivas

    return response
    

In [53]:
best_developer_year(2017)

{'Puesto 1': 'Smartly Dressed Games',
 'Puesto 2': 'Freejam',
 'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement Llc'}

# Endpoint 5

In [49]:
def developer_reviews_analysis(desarrolladora: str):
    df = pd.read_parquet('endpoint45', engine='auto') #Leemos el dataset

    df.drop(columns=['recommend_True', 'release_date']) #Borramos columnas innecesarias
    df = df[(df['developer'] == desarrolladora.lower())] #Lo filtramos por el valor ingresado
    if df.empty: #Si no devuelve resultados, termina la función
        return "No se encontraron datos"
    df = df.groupby(['developer']).sum(['sent_pos', 'sent_neg']) #Sumamos reviews positivas y negativas por el desarrollador
    
    neg = f"Negativo = {df['sent_neg'][0]}" #Creamos variables con los valores a devolver
    pos = f"Positivo = {df['sent_pos'][0]}"

    response = {desarrolladora.title(): [neg, pos]} #Generamos la respuesta

    return response

In [50]:
developer_reviews_analysis('saber interactive')

{'Saber Interactive': ['Negativo = 0', 'Positivo = 3']}

# Sistema de recomendacion de juegos por juego

In [26]:
def recomendacion_juego(idproducto : int):
    df = pd.read_parquet('recom_juegos',engine='auto') #Leemos el dataset
    juego = df[(df['item_id'] == idproducto)] #Lo filtramos según el dato ingresado

    if juego.empty: #Si no se encuentran valores, termina la función
        return "No se encontraron datos"
    
    r = f"Juegos recomendados similares a {juego['titles'].iloc[0].capitalize()}" #Generamos la primera etiqueta de respuesta
    df = df[((df['main_genre'] == juego['main_genre'].iloc[0]))].reset_index() #Filtramos el dataframe segun el genero del juego localizado
    indice = df.index[df['titles'] == juego['titles'].iloc[0]][0] #Obtenemos el indice del juego ingresado
    df['score'] = df['sent_pos'] - df['sent_neg'] # Generamos nueva columna con un puntaje donde a las reviews positivas se le restan las negativas
    vector = TfidfVectorizer() #Instansiamos el vectorizador
    cosine = cosine_similarity(vector.fit_transform(df['genres'])) #Aplicamos el coseno de similitud sobre la vectorización de la columna de referencia sobre las que se van a calcular las similitudes
    df['similarity'] = list(cosine[indice])  #Creamos una columna con el coseno de similitud calculado en funcion del juego con ese indice
    recs = df.sort_values(['similarity', 'score'], ascending=False)[1:6] # Ordenamos los valores según similitud y puntaje y extraemos los primeros 5,excluyendo el juego ingresado
    recs['titles'] = recs['titles'].apply(lambda x: x.capitalize()) # Le ponemos la primera letra mayuscula a los titulos extraidos

    response = {r : recs['titles'].tolist()} #Generamos respuesta

    return response
    

In [31]:
recomendacion_juego(658870)

{'Juegos recomendados similares a Exit 2 - directions': ['Exit',
  'Perso',
  'Puzzler',
  'Planktos',
  'Wayout 2: hex']}

# Sistema de recomendación de juegos por usuario

In [44]:
def recomendacion_usuario(id_de_usuario : str):
    df = pd.read_parquet('recom_juegos2')
    pref = df[(df['user_id'] == id_de_usuario.lower())].sort_values(by='playtime_forever', ascending=False)[:1] #localizamos el juego que el usuario jugó mas

    if pref.empty: #Si no encuentra datos, termina la función
        return 'No se encontraron datos'
    
    r = f"Juegos recomendados para el usuario {id_de_usuario.capitalize()}" #Generamos la primera etiqueta de respuesta
    df = df.drop_duplicates(subset='titles', keep='last') #Sacamos del dataframe los valores repetidos
    df = df[((df['main_genre'] == pref['main_genre'].iloc[0]))].reset_index() #Filtramos el dataframe segun el genero principal del juego localizado
    indice = df.index[df['titles'] == pref['titles'].iloc[0]][0] #Obtenemos el indice del juego preferido del usuario
    df['score'] = df['sent_pos'] - df['sent_neg'] # Generamos nueva columna con un puntaje donde a las reviews positivas se le restan las negativas
    vector = TfidfVectorizer() #Instansiamos el vectorizador
    cosine = cosine_similarity(vector.fit_transform(df['genres'])) #Aplicamos el coseno de similitud sobre la vectorización de la columna de referencia sobre las que se van a calcular las similitudes
    df['similarity'] = list(cosine[indice])  #Creamos una columna con el coseno de similitud calculado en funcion del juego con ese indice
    

    recs = df.sort_values(['similarity', 'score'], ascending=False)[1:6] # Ordenamos los valores según similitud y puntaje y extraemos los primeros 5,excluyendo el juego ingresado
    recs['titles'] = recs['titles'].apply(lambda x: x.capitalize()) # Le ponemos la primera letra mayuscula a los titulos extraidos

    response = {r : recs['titles'].tolist()} #Generamos respuesta

    return response
        

In [48]:
recomendacion_usuario('gamerz_united')

{'Juegos recomendados para el usuario Gamerz_united': ['Simpleplanes',
  'Mount & blade',
  "Tiny and big: grandpa's leftovers",
  'Xsyon - prelude',
  'Rimworld']}